In [1]:
from neo4j import GraphDatabase
import DATABASE_CONFIG

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j://localhost"
AUTH = ("neo4j", DATABASE_CONFIG.DB_PASSWORD)

In [2]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    driver.execute_query(
        "CREATE CONSTRAINT id_estacao_meteorologica_constraint FOR (r:EstacaoMeteorologica) REQUIRE r.id_estacao_meteorologica IS UNIQUE"
    )
    driver.execute_query(
        "CREATE CONSTRAINT nome_estacao_meteorologica_constraint FOR (r:EstacaoMeteorologica) REQUIRE r.nome IS UNIQUE"
    )


In [3]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    driver.execute_query(
        "LOAD CSV WITH HEADERS FROM 'file:///estacao_meteorologica.csv' AS csvLine "
        "CREATE (r:EstacaoMeteorologica {id_estacao_meteorologica: toInteger(csvLine.id_estacao_meteorologica), nome: csvLine.nome, latitude: toFloat(csvLine.latitude), longitude: toFloat(csvLine.longitude), altitude: toFloat(csvLine.altitude), data_fundacao: date(csvLine.data_fundacao)})"
    )

In [4]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    driver.execute_query(
        "LOAD CSV WITH HEADERS FROM 'file:///estacao_meteorologica.csv' AS csvLine "
        "MATCH (e:EstacaoMeteorologica {id_estacao_meteorologica: toInteger(csvLine.id_estacao_meteorologica)}) "
        "MATCH (c:Cidade {id_cidade: toInteger(csvLine.id_cidade)}) "
        "MERGE (e)-[:Pertence]->(c) "
        )

In [8]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    driver.execute_query(
        """
        LOAD CSV WITH HEADERS FROM 'file:///relatorio_meteorologia.csv' AS csvLine
        MATCH (e:EstacaoMeteorologica {id_estacao_meteorologica: toInteger(csvLine.id_estacao_meteorologica)})-[:Pertence]->(c:Cidade)
        CREATE (e)-[d:Dados_Meteorologicos {
            id_meteorologia: toInteger(csvLine.id_meteorologia),
            data_horario_coleta: datetime(replace(csvLine.data_horario_coleta, ' ', 'T')),
            temperatura_max: toFloat(csvLine.temperatura_max),
            temperatura_min: toFloat(csvLine.temperatura_min),
            precipitacao: toFloat(csvLine.precipitacao),
            umidade: toFloat(csvLine.umidade),
            pressao_atm: toFloat(csvLine.pressao_atm),
            vento_velocidade: toFloat(csvLine.vento_velocidade)
        }]->(c)
        """
        )